## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,75.72,25,75,17.27,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,33.66,86,84,12.26,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,48.31,49,0,18.41,clear sky
3,3,Saint-Philippe,RE,-21.3585,55.7679,69.39,70,51,9.08,broken clouds
4,4,Busselton,AU,-33.6500,115.3333,54.50,72,100,9.26,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 89


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,75.72,25,75,17.27,broken clouds
6,6,Atuona,PF,-9.8000,-139.0333,78.26,73,62,15.21,broken clouds
7,7,Kapaa,US,22.0752,-159.3190,82.38,71,75,11.50,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,80.76,74,20,4.61,few clouds
11,11,Iberia,US,29.9502,-91.7507,88.05,65,0,9.22,thunderstorm
12,12,Wewak,PG,-3.5534,143.6268,81.10,77,39,4.36,scattered clouds
20,20,Port Blair,IN,11.6667,92.7500,78.55,87,100,9.24,overcast clouds
25,25,Noumea,NC,-22.2763,166.4572,75.18,99,0,1.63,clear sky
33,33,Bethel,US,41.3712,-73.4140,75.36,34,0,9.22,clear sky
35,35,Vieste,IT,41.8825,16.1750,80.11,52,96,11.43,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                204
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yellowknife,CA,75.72,broken clouds,62.4560,-114.3525,
6,Atuona,PF,78.26,broken clouds,-9.8000,-139.0333,
7,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,
8,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
11,Iberia,US,88.05,thunderstorm,29.9502,-91.7507,
12,Wewak,PG,81.10,scattered clouds,-3.5534,143.6268,
20,Port Blair,IN,78.55,overcast clouds,11.6667,92.7500,
25,Noumea,NC,75.18,clear sky,-22.2763,166.4572,
33,Bethel,US,75.36,clear sky,41.3712,-73.4140,
35,Vieste,IT,80.11,overcast clouds,41.8825,16.1750,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df["Hotel Name"] == ""]
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yellowknife,CA,75.72,broken clouds,62.4560,-114.3525,The Explorer Hotel
6,Atuona,PF,78.26,broken clouds,-9.8000,-139.0333,Villa Enata
7,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Iberia,US,88.05,thunderstorm,29.9502,-91.7507,back to back rv park


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))